<a href="https://colab.research.google.com/github/viyaleta/DES/blob/master/DES_Timing_Wheel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class Object:
    def __init__(self, name, init_state, init_state_duration=1):
        self.name = name
        self.current_state = init_state
        self.current_state_duration = init_state_duration
        self.future_events = [] # tuple pairs of [state, duration]

    def update_state(self):
        # if nothing scheduled for the future, return -1
        if len(self.future_events) == 0:
            return 0
        
        # get state out of future events and set it as current state
        # also pop it out of future events
        new_state, new_state_duration = self.future_events.pop(0)
        self.current_state = new_state
        self.current_state_duration = new_state_duration
        print('{} state changed to {} for {} ticks'
              .format(self.name, self.current_state, self.current_state_duration))
        return self.current_state_duration

    def get_next_event(self):
        if len(self.future_events)>0:
            return self.future_events[0][1]
        else:
            return -1

In [0]:
class Wheel:
    def __init__(self, objects=[], ticks=60):
        self.ticks = ticks
        self.wheel_turn = 0
        self.current_time = -1
        self.objects = objects
        self.slots = None
        self.future_slots = []
        self.__initialize_wheel()

    def __initialize_wheel(self):
        # initialize slots
        self.__initialize_slots()
        
        # set time to 1 = begin
        self.__tick_tock()

        for object in self.objects:
            # add object to slot of "current time + duration of current state of this object"
            self.slots[self.current_time+object.current_state_duration].append(object)
            print('{} - {} added to time wheel'.format(object.name, object.current_state))
            self.__process_queue()
            
    def __initialize_slots(self, future=False):
        # if initializing a future slot, append it to future_slots list
        if future:
            self.future_slots.append({i:[] for i in range(self.ticks)})
            
        # if initializing current slot and there is nothing to transfer from future
        elif len(self.future_slots)==0:
            self.slots = {i:[] for i in range(self.ticks)}
            
        # if transfering a future slot to current wheel turn
        else:
            self.slots = self.future_slots.pop(0)

    def simulate(self):
        while True:            
            if self.current_time < self.ticks - 1:
                # add 1 to current time and process queue
                self.__tick_tock()
                self.__process_queue()
            elif len(self.objects)>0:
                # if the queue is not empty
                # turn the wheel and reset the ticks
                self.wheel_turn += 1
                self.current_time = -1
                self.__initialize_slots()
            else:
                # nothing left to do - terminate the simulation
                break

    def __tick_tock(self):
        self.current_time += 1
        if len(self.slots[self.current_time])>0:
            print('Wheel:', self.wheel_turn,'Time:', self.current_time)

    def __process_queue(self):
        ''' Function to process queue at current time
        1. Update state of each object that is present in the slot of current time
        2. For each object presentSchedule future events
        
        * If the wheel doesn't have enough ticks, schedule to future slot
        ''' 

        # schedule future event for each object
        for object in self.slots[self.current_time]:
            
            # update state
            current_state_duration = object.update_state()

            if current_state_duration == 0:
                self.objects.remove(object)
                print('Simulation for {} complete'.format(object.name))
                continue  
                # skip the rest because we don't need to schedule anything
                
            # calculate next event time = now + how long current state will last
            next_event_time = self.current_time + current_state_duration
            
            # add object to the slot of next event time or complete journey
            if next_event_time <= self.ticks:
                self.slots[next_event_time].append(object)
            
            # if next event time > wheel capactiy, schedule to next future
            else:
                # how many wheels ahead to we need to schedule?
                turn_count = next_event_time // self.ticks
                
                # check to see if future slot exists, if not initialize it
                for i in range(len(self.future_slots), turn_count):
                    self.__initialize_slots(future=True)
                
                next_event_time = next_event_time % self.ticks

                # add object to proper wheel turn number and slot number
                self.future_slots[turn_count-1][next_event_time].append(object)

        self.slots[self.current_time] = []


In [65]:
# Simple Car Simulation

car = Object('Car', 'idle', 1)
car.future_events = [
    ('traveling', 4),
    ('getting gas', 3),
    ('traveling', 20),
    ('arriving', 1)
]

tw = Wheel([car], 60)
tw.simulate()

# Car state changed to traveling - should be at 1
# Car state changed to getting gas - should be at 5
# Car state changed to traveling - should be at 8
# Car state changed to arriving - should be at 28
# Car simulation complete - should be at 29

Car - idle added to time wheel
Wheel: 0 Time: 1
Car state changed to traveling for 4 ticks
Wheel: 0 Time: 5
Car state changed to getting gas for 3 ticks
Wheel: 0 Time: 8
Car state changed to traveling for 20 ticks
Wheel: 0 Time: 28
Car state changed to arriving for 1 ticks
Wheel: 0 Time: 29
Simulation for Car complete
